### 搭建第一个全连接网络训练MNIST准确率>85%的模型（PyTorch）

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

#### 1. 定义数据转换和加载器
使用`transforms`对数据进行预处理，并创建数据加载器。

In [ ]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=1000, shuffle=False)

#### 2. 定义全连接神经网络模型
使用`nn.Module`定义一个简单的全连接神经网络。

In [ ]:
class SimpleNN(nn.Module):#nn.Module 是 PyTorch 中所有神经网络模块的基类。
    def __init__(self):
        super(SimpleNN, self).__init__()#调用父类 (nn.Module) 的构造函数来初始化父类中的一些属性。
        self.fc1 = nn.Linear(28*28, 128)#线性层 输入 28*28 并转换为 128的输出
        self.fc2 = nn.Linear(128, 64) #第二个全连接层 fc2。接收 128 的输入，转换为64 的输出
        self.fc3 = nn.Linear(64, 10) #接收64 的输入，转换为10 的输出。
        #由于 MNIST 数据集有 10 个类别（数字 0 到 9）

    def forward(self, x):#输入数据如何通过网络的各个层进行计算
        x = x.view(-1, 28*28) 
        x = torch.relu(self.fc1(x)) #ReLU 函数将所有负值替换为 0，并保留正值
        x = torch.relu(self.fc2(x)) 
        x = self.fc3(x)
        return x

model = SimpleNN()
#这一行创建了一个 SimpleNN 类的实例，并将其赋值给变量 model。现在，model 就代表了我们定义的全连接神经网络。

#### 3. 定义损失函数和优化器
使用交叉熵损失函数和Adam优化器。

In [4]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

#### 4. 训练模型
训练模型并在每个epoch后打印损失。

In [5]:
epochs = 10
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        outputs = model(data)
        loss = criterion(outputs, target)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f'Epoch [{epoch+1}/{epochs}], Loss: {running_loss/len(train_loader):.4f}')

#### 5. 测试模型
在测试数据集上评估模型的准确率。

In [6]:
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for data, target in test_loader:
        outputs = model(data)
        _, predicted = torch.max(outputs.data, 1)
        total += target.size(0)
        correct += (predicted == target).sum().item()
print(f'Test Accuracy: {100 * correct / total:.2f}%')

Test Accuracy: 97.83%
